<a href="https://colab.research.google.com/github/erwanBellon/2025_ML_EES/blob/main/project/code/XAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Feature map



#### I.1 : setup

In [ ]:
# @title

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Cloning repo or fetch latest changes and path management
!git clone https://github.com/erwanBellon/2025_ML_EES.git
%cd /content/2025_ML_EES
!git pull

import os
from pathlib import Path

# Move into the project directory
%cd /content/2025_ML_EES/project/code
print("Current working directory:", Path.cwd())

# Define main project dir and outputs
PROJECT_ROOT_DIR = Path.cwd().parent       # -> /content/2025_ML_EES/project
OUTPUTS_PATH = PROJECT_ROOT_DIR / "outputs"
OUTPUTS_PATH.mkdir(parents=True, exist_ok=True)
print("Outputs will be saved to:", OUTPUTS_PATH)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU detected. CNNs can be slow without GPU.")

# Common imports
import pandas as pd
import numpy as np
!pip install rasterio
import rasterio
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# To make notebook reproducible
np.random.seed(42)
tf.random.set_seed(42)

# For plots
import matplotlib.pyplot as plt
%matplotlib inline

# Load Tensorboard
%load_ext tensorboard

Cloning into '2025_ML_EES'...
remote: Enumerating objects: 1018, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 1018 (delta 183), reused 51 (delta 51), pack-reused 769 (from 2)
Receiving objects: 100% (1018/1018), 256.90 MiB | 9.61 MiB/s, done.
Resolving deltas: 100% (669/669), done.
Updating files: 100% (366/366), done.
/content/2025_ML_EES
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 12), reused 1 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (21/21), 6.52 KiB | 607.00 KiB/s, done.
From https://github.com/erwanBellon/2025_ML_EES
   4c8eb3f..d61330e  main       -> origin/main
Updating 4c8eb3f..d61330e
Fast-forward
 project/code/XAI.ipynb                            | 154 ++++++++++++--------
 project/code/model_habitat_amount_3000_v0.ipynb   | 168 +++++++++++++---------
 project/outputs/{ => 

In [ ]:
print(Path.cwd())

/content/2025_ML_EES/project/code


## I.2 Load the model using a CNN-ANN framework

In [ ]:
# @title
# Path to your saved model
model_path = Path.cwd() /"../outputs/cnn_bestModel.keras"

# Load model
model = load_model(model_path)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 30, 30, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_flip         │ (None, 30, 30, 1) │          0 │ image_input[0][0] │
│ (RandomFlip)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_zoom         │ (None, 30, 30, 1) │          0 │ random_flip[0][0] │
│ (RandomZoom)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_translation  │ (None, 30, 30, 1) │          0 │ random_zoom[0][0] │
│ (RandomTranslation) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 30, 30,    │        320 │ random_translati… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 30, 30,    │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 15, 15,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 15, 15,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 7, 7, 64)  │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ table_input         │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3136)      │          0 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │         48 │ table_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │    100,384 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8)         │        136 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 40)        │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      1,312 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 8)         │        264 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │          9 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 390,653 (1.49 MB)

 Trainable params: 130,217 (508.66 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 260,436 (1017.33 KB)

### I.3 Load image data (and table input) and preprocess the datas


In [ ]:
# @title
# --- Load images ---
presences_path = Path("../data/cropped_landcover/presences")
absences_path = Path("../data/cropped_landcover/absences")

def load_images_from_folder(folder):
    tif_files = list(folder.glob("*.tif"))
    images = []
    image_indices = []
    for tif in tif_files:
        with rasterio.open(tif) as src:
            img = src.read()
            img = np.transpose(img, (1,2,0))
            images.append(img.astype(np.float32))
        # Extract the line index from the filename (assuming `crop_3000_118.tif`)
        idx = int(tif.stem.split("_")[-1])
        image_indices.append(idx)
    return np.array(images), np.array(image_indices)

images_pres, indices_pres = load_images_from_folder(presences_path)
images_abs, indices_abs = load_images_from_folder(absences_path)
print(f"Presences: {images_pres.shape}, Absences: {images_abs.shape}")

# Build dataset & labels
X = np.concatenate([images_pres, images_abs], axis=0)
y = np.concatenate([np.ones(len(images_pres)), np.zeros(len(images_abs))], axis=0).astype(np.int32)
image_indices = np.concatenate([indices_pres, indices_abs], axis=0)  # all image row indices

# --- Load table data ---
rds_path = Path("../data/Table_preds/function_3_100.rds")
!pip install pyreadr
import pyreadr
result = pyreadr.read_r(rds_path)
table_df = result[None]  # get DataFrame

# Select only the rows corresponding to actual images
table_features_all = table_df[['MAP','MAT']].astype(float)
table_features = table_features_all.iloc[image_indices].reset_index(drop=True)

# Normalize
table_features = (table_features - table_features.min()) / (table_features.max() - table_features.min())
table_features = table_features.to_numpy(dtype=np.float32)

X_train, X_temp, table_train, table_temp, y_train, y_temp = train_test_split(
    X, table_features, y, test_size=0.2, random_state=42, stratify=y
)
X_valid, X_test, table_valid, table_test, y_valid, y_test = train_test_split(
    X_temp, table_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)



Presences: (117, 30, 30, 1), Absences: (200, 30, 30, 1)


In [ ]:
# @title
train_ds = tf.data.Dataset.from_tensor_slices(((X_train, table_train), y_train))
valid_ds = tf.data.Dataset.from_tensor_slices(((X_valid, table_valid), y_valid))
test_ds  = tf.data.Dataset.from_tensor_slices(((X_test, table_test), y_test))


# Add `.name` attribute like TFDS
train_ds.name = "Training"
valid_ds.name = "Validation"
test_ds.name  = "Test"

In [ ]:
# @title
def preprocess_multi_inputs(inputs, label):
    image, table = inputs
    # Convert image to float32 and add channel dimension if needed
    image = tf.cast(image, tf.float32)
    # Replace Nan (non-forest) with 0s
    image = tf.where(tf.math.is_nan(image), 0.0, image)

    if len(image.shape) == 3:  # (H,W,C) or (H,W)
        image = tf.expand_dims(image, -1)  # ensures (H,W,1)
    # Table should already be (batch_size, 2) after batching
    table = tf.cast(table, tf.float32)
    label = tf.cast(label, tf.float32)
    return (image, table), label

In [ ]:
# @title
train_ds = (
    tf.data.Dataset.from_tensor_slices(((X_train, table_train), y_train))
      .shuffle(1000, reshuffle_each_iteration=True)   # <--- SHUFFLE HERE
      .map(preprocess_multi_inputs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE)
)
valid_ds = (
    tf.data.Dataset.from_tensor_slices(((X_valid, table_valid), y_valid))
      .map(preprocess_multi_inputs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices(((X_test, table_test), y_test))
      .map(preprocess_multi_inputs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE)
)

# Add `.name` attribute like TFDS
train_ds.name = "Training"
valid_ds.name = "Validation"
test_ds.name  = "Test"


Use the test dataset and prepare 15 sample images


In [ ]:
# @title
sample_images = []
sample_tables = []

for (img, tab), label in test_ds.take(15):  # to get single samples
    # Image preprocessing
    img = tf.cast(img, tf.float32)
    img = tf.where(tf.math.is_nan(img), 0.0, img)
    if len(img.shape) == 2:  # (H,W)
        img = tf.expand_dims(img, -1)  # (H,W,1)
    sample_images.append(img.numpy())

    # Table preprocessing
    tab = tf.cast(tab, tf.float32)
    sample_tables.append(tab.numpy())

# Convert to numpy arrays with batch dimension
sample_images = np.stack(sample_images)
sample_tables = np.stack(sample_tables)

print("Sample images shape:", sample_images.shape)   # (5,H,W,C)
print("Sample tables shape:", sample_tables.shape)   # (5,n_features)
print("Min/Max values:", img.numpy().min(), img.numpy().max())



Sample images shape: (15, 30, 30, 1, 1)
Sample tables shape: (15, 2)
Min/Max values: 0.0 0.0


## I.4. Looking at the feature maps

In [ ]:
# @title
# Print all layers with their output shapes
for i, layer in enumerate(model.layers):
    try:
        shape = layer.output.shape
    except AttributeError:
        shape = "N/A"  # For InputLayer or unusual layers
    print(i, layer.name, shape, type(layer))


0 image_input (None, 30, 30, 1) <class 'keras.src.layers.core.input_layer.InputLayer'>
1 random_flip (None, 30, 30, 1) <class 'keras.src.layers.preprocessing.image_preprocessing.random_flip.RandomFlip'>
2 random_zoom (None, 30, 30, 1) <class 'keras.src.layers.preprocessing.image_preprocessing.random_zoom.RandomZoom'>
3 random_translation (None, 30, 30, 1) <class 'keras.src.layers.preprocessing.image_preprocessing.random_translation.RandomTranslation'>
4 conv2d (None, 30, 30, 32) <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
5 conv2d_1 (None, 30, 30, 32) <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
6 max_pooling2d (None, 15, 15, 32) <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
7 conv2d_2 (None, 15, 15, 64) <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
8 max_pooling2d_1 (None, 7, 7, 64) <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
9 table_input (None, 2) <class 'keras.src.layers.core.input_layer.InputLayer'>
10 flatten (None, 

#### Select the part of the model that outputs features

I'm interested at looking at the feature map of the last convolution layer (conv2d_2). So I extract it and output its feature (sub_model)

In [ ]:
# @title

# Get the conv layer of interest
conv_layer = model.get_layer("conv2d_2")
print("conv layer:",conv_layer.name, conv_layer.output.shape)

# Build sub-model from original model ---
# Input = both image and table, output = last conv layer
feature_map_model = Model(inputs=model.inputs, outputs=conv_layer.output)

# Prepare 15 samples from test dataset ---
sample_images = []
sample_tables = []

for (img, tab), label in test_ds.take(15):  # get single samples
    # Image preprocessing
    img = tf.cast(img, tf.float32)
    img = tf.where(tf.math.is_nan(img), 0.0, img)
    if len(img.shape) == 2:  # (H,W)
        img = tf.expand_dims(img, -1)  # (H,W,1)
    sample_images.append(img.numpy())

    # Table preprocessing
    tab = tf.cast(tab, tf.float32)
    sample_tables.append(tab.numpy())

# Convert to numpy arrays with batch dimension
sample_images = np.stack(sample_images)
sample_tables = np.stack(sample_tables)

print("Sample images shape:", sample_images.shape)   # (15,H,W,C)
print("Sample tables shape:", sample_tables.shape)   # (15,n_features)
print("Min/Max values:", img.numpy().min(), img.numpy().max())




conv layer: conv2d_2 (None, 15, 15, 64)
Sample images shape: (15, 30, 30, 1, 1)
Sample tables shape: (15, 2)
Min/Max values: 0.0 0.0


In [ ]:
# @title
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Custom colormap: 0 = white, 1 = dark green
cmap_img = ListedColormap(['white', 'darkgreen'])

# Map label to text
label_map = {1: "Presence", 0: "Absence"}

# Directory to save plots in Colab
save_dir = Path("/content/outputs")
save_dir.mkdir(parents=True, exist_ok=True)

for i in range(len(sample_images)):
    img_batch = sample_images[i:i+1]   # shape (1,H,W,C)
    tab_batch = sample_tables[i:i+1]   # shape (1,n_features)
    label = y_test[i]                   # true label

    # --- Get model prediction ---
    pred_prob = model.predict([img_batch, tab_batch], verbose=0)[0,0]

    # Predict feature maps
    feature_maps = feature_map_model.predict([img_batch, tab_batch], verbose=0)

    # Compute mean activation per filter
    mean_activation = feature_maps.mean(axis=(1,2))  # shape (1, n_filters)

    # Get top 3 activated filters
    top3_idx = np.argsort(mean_activation[0])[::-1][:3]

    # Plot: original + 3 feature maps
    fig, axes = plt.subplots(1, 4, figsize=(16,5))

    # Original image with predicted probability
    axes[0].imshow(img_batch[0, :, :, 0], cmap=cmap_img, vmin=0, vmax=1)
    axes[0].set_title(f"Original Image\nTrue: {label_map[label]}, Pred: {pred_prob:.3f}")
    axes[0].axis("off")

    # Feature maps
    for j, idx in enumerate(top3_idx):
        activation = feature_maps[0, :, :, idx]
        activation = (activation - activation.min()) / (activation.max() - activation.min() + 1e-8)
        im = axes[j+1].imshow(activation, cmap="viridis")
        axes[j+1].set_title(f"Filter {idx}")
        axes[j+1].axis("off")
        fig.colorbar(im, ax=axes[j+1], fraction=0.046, pad=0.04)

    # Save figure
    save_path = save_dir / f"sample_{i}_{label_map[label]}.png"
    plt.savefig(save_path, bbox_inches="tight")
    plt.close(fig)

print(f"Plots saved to {save_dir}")


Plots saved to /content/outputs


## Part II: Permutation feature importance (can be ran independently of the rest)

### II.1 Permutation importance for CNN-ANN

Large drop in AUC after shuffling→ feature is important

Small or no drop → feature is less important

Negative drop → shuffling actually improved performance (can happen due to noise or correlations)

In [ ]:

from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import rasterio
import pyreadr

#1. Load and preprocess images
def load_images_from_folder(folder):
    tif_files = list(folder.glob("*.tif"))
    images = []
    image_indices = []
    for tif in tif_files:
        with rasterio.open(tif) as src:
            img = src.read()
            img = np.transpose(img, (1, 2, 0))  # (H,W,C)
            img = img.astype(np.float32)
            img = np.where(np.isnan(img), 0.0, img)  # replace NaNs
            if img.ndim == 2:
                img = np.expand_dims(img, -1)
            images.append(img)
        idx = int(tif.stem.split("_")[-1])
        image_indices.append(idx)
    return np.array(images), np.array(image_indices)

presences_path = Path("../data/cropped_landcover/presences")
absences_path = Path("../data/cropped_landcover/absences")

images_pres, indices_pres = load_images_from_folder(presences_path)
images_abs, indices_abs = load_images_from_folder(absences_path)

# Labelsadding
X = np.concatenate([images_pres, images_abs], axis=0)
y = np.concatenate([np.ones(len(images_pres)), np.zeros(len(images_abs))], axis=0)
image_indices = np.concatenate([indices_pres, indices_abs], axis=0)

# 2. Load table data
rds_path = Path("../data/Table_preds/function_3_100.rds")
result = pyreadr.read_r(rds_path)
table_df = result[None]

table_features_all = table_df[['MAP','MAT']].astype(float)
table_features = table_features_all.iloc[image_indices].reset_index(drop=True)
# normalize
table_features = (table_features - table_features.min()) / (table_features.max() - table_features.min())
table_features = table_features.to_numpy(dtype=np.float32)

#  3. Split dataset
X_train, X_temp, table_train, table_temp, y_train, y_temp = train_test_split(
    X, table_features, y, test_size=0.2, random_state=42, stratify=y
)
X_valid, X_test, table_valid, table_test, y_valid, y_test = train_test_split(
    X_temp, table_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

#  4. Load trained model
model1_path = Path.cwd() / "../outputs/cnn_bestModel.keras"
model1 = load_model(model1_path)


#  5. Build inference model
from tensorflow import keras

def build_inference_model(model_trained):
    # Inputs
    image_input = keras.Input(shape=(30,30,1), name="image_input")
    table_input = keras.Input(shape=(2,), name="table_input")

    # Image branch ---
    x = keras.layers.Conv2D(32, (3,3), activation="relu", padding="same", name="conv2d")(image_input)
    x = keras.layers.Conv2D(32, (3,3), activation="relu", padding="same", name="conv2d_1")(x)
    x = keras.layers.MaxPooling2D((2,2), name="max_pooling2d")(x)
    x = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same", name="conv2d_2")(x)
    x = keras.layers.MaxPooling2D((2,2), name="max_pooling2d_1")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    x = keras.layers.Dense(32, activation="sigmoid", name="dense")(x)

    # Table branch
    t = keras.layers.Dense(16, activation="relu", name="dense_1")(table_input)
    t = keras.layers.Dense(8, activation="relu", name="dense_2")(t)

    # Combined
    merged = keras.layers.Concatenate(name="concatenate")([x, t])
    x = keras.layers.Dense(32, activation="relu", name="dense_3")(merged)
    x = keras.layers.Dropout(0.4, name="dropout")(x, training=False)
    x = keras.layers.Dense(8, activation="relu", name="dense_4")(x)
    output = keras.layers.Dense(1, activation="sigmoid", name="dense_5")(x)

    inference_model = keras.Model(inputs=[image_input, table_input], outputs=output)

    # Copy weights from trained model by layer name
    for layer in inference_model.layers:
        try:
            trained_layer = model_trained.get_layer(layer.name)
            layer.set_weights(trained_layer.get_weights())
        except ValueError:
            # Skip layers that do not exist (e.g., new Input layers)
            pass

    return inference_model


model1_infer = build_inference_model(model1)

# 6. Permutation importance
# Shuffle the features randomly and evaluate the drop (or delta)in AUC
def permutation_importance_model1(model, images, table, y, feature_names=None, n_repeats=20):

    images = np.asarray(images, dtype=np.float32)
    table = np.asarray(table, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)

    if feature_names is None:
        feature_names = [f"Feature_{i}" for i in range(table.shape[1])]

    # Baseline prediction (i.e. without shuffling)
    base_pred = model.predict([images, table], verbose=0).flatten()
    base_auc = roc_auc_score(y, base_pred)
    print(f"Baseline AUC: {base_auc:.4f}")

    importances = {}

    # Table features
    for col in range(table.shape[1]):
        scores = []
        for _ in range(n_repeats):
            table_perm = table.copy()
            np.random.shuffle(table_perm[:, col])  # shuffle this column
            pred = model.predict([images, table_perm], verbose=0).flatten()
            auc = roc_auc_score(y, pred)
            scores.append(base_auc - auc)
        importances[feature_names[col]] = np.mean(scores)


    # Images
    scores = []
    for _ in range(n_repeats):
        images_perm = images.copy()
        np.random.shuffle(images_perm)  # shuffle entire images
        pred = model.predict([images_perm, table], verbose=0).flatten()
        auc = roc_auc_score(y, pred)
        scores.append(base_auc - auc)
    importances["Image"] = np.mean(scores)

    # Results print
    print("\nPermutation Feature Importances:")
    for key, value in sorted(importances.items(), key=lambda x: abs(x[1]), reverse=True):
        print(f"  {key:20s}: {value:+.4f}")

    return importances, base_auc





#  7. Compute importance
table_feature_names = ['MAP', 'MAT']
importance, base_auc = permutation_importance_model1(
    model1_infer,
    X_test,
    table_test,
    y_test,
    feature_names=table_feature_names,
    n_repeats=5
)




Baseline AUC: 0.8417

Permutation Feature Importances:
  Image               : +0.2542
  MAT                 : +0.0300
  MAP                 : -0.0117


Note that the base AUC is slightly different from the model evaluation because I had to remove the data-augmentation layers when doing inference for feature permutation

#### II.2 Permutation for ANN only

In [ ]:

import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import load_model
import pyreadr

# Load table data for ANN-only
rds_path = Path("../data/Table_preds/function_3_100.rds")
result = pyreadr.read_r(rds_path)
table_df = result[None]

# Select relevant columns
feature_df = table_df[['MAP','MAT','habitat_amount_3000']].astype(float)

# Load indices for presences/absences
presences_path = Path("../data/cropped_landcover/presences")
absences_path = Path("../data/cropped_landcover/absences")

def extract_indices(folder):
    tif_files = list(folder.glob("*.tif"))
    indices = []
    for tif in tif_files:
        idx = int(tif.stem.split("_")[-1])
        indices.append(idx)
    return np.array(indices)

indices_pres = extract_indices(presences_path)
indices_abs = extract_indices(absences_path)

# Build label vector
y = np.concatenate([
    np.ones(len(indices_pres)),
    np.zeros(len(indices_abs))
]).astype(np.float32)

# Extract corresponding rows
feature_all = feature_df
features = feature_all.iloc[np.concatenate([indices_pres, indices_abs])]
features = features.reset_index(drop=True)

# Normalize features 0–1
features = (features - features.min()) / (features.max() - features.min())
features = features.to_numpy(np.float32)

# Train/valid/test split
X_train, X_temp, y_train, y_temp = train_test_split(
    features, y, test_size=0.2, random_state=42, stratify=y
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Load ANN-only modeol
model2_path = Path.cwd() / "../outputs/HA_3000_bestModel_v2.keras"
model2 = load_model(model2_path)

# Permutation importance for table features

def permutation_importance_model2(model, table, y, feature_names=None, n_repeats=10):

    table = np.asarray(table, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)

    if feature_names is None:
        feature_names = [f"Feature_{i}" for i in range(table.shape[1])]

    # Base prediction & AUC
    base_pred = model.predict(table, verbose=0).flatten()
    base_auc = roc_auc_score(y, base_pred)
    print(f"Baseline AUC: {base_auc:.4f}")

    importances = {}
    for col in range(table.shape[1]):
        scores = []
        for _ in range(n_repeats):
            table_perm = table.copy()
            np.random.shuffle(table_perm[:, col])
            pred = model.predict(table_perm, verbose=0).flatten()
            auc = roc_auc_score(y, pred)
            scores.append(base_auc - auc)
        importances[feature_names[col]] = np.mean(scores)

    # Result print
    print("\nPermutation importances:")
    for k, v in sorted(importances.items(), key=lambda x: -x[1]):
        print(f"{k}: {v:.4f}")

    return importances

# Compute permutation importances on test set
feature_names = ['MAP', 'MAT', 'habitat_amount_3000']
importance = permutation_importance_model2(model2, X_test, y_test, feature_names=feature_names, n_repeats=5)


Baseline AUC: 0.6833

Permutation importances:
MAP: 0.0350
MAT: 0.0058
habitat_amount_3000: -0.0100


The change is baseline AUC may also be due to the dataleakage during the pre-processing of the data..